In [2]:
#Import Required Libraries
from PIL import Image
import requests

In [3]:
# Ensures that there is enough memory allocation for the model to load
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
from transformers import pipeline
import torch

pipe = pipeline(
    "image-text-to-text",
    model="google/gemma-3-4b-it",
    device="cuda",
    torch_dtype=torch.bfloat16
)


/home/jovyan/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.20it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


In [ ]:
# Input Image
url = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"

# File path to save the questions
file_path = 'url.txt'

# Writing the questions string to a txt file
with open(file_path, 'w') as file:
    file.write(url)  # Strip any leading/trailing whitespace

print(f"Questions have been saved to {file_path}")


image = Image.open(requests.get(url, stream=True).raw)
image

In [6]:
# Number of questions to be generated 
questions = 20

In [13]:
prompt = f"""
Task : Generate a list of {questions} simple questions for each of the following three topics related to object detection for other Visual Language Models to infer:

Instructions :
1. **Classes of Object Detection**: Ask questions about the categories or types of objects detected in the image (e.g., "What is the object class?" or "Which class is the object?").
2. **Number of Objects**: Ask questions related to how many objects are detected in an image (e.g., "How many objects are in the image?" or "What is the count of objects?").
3. **Colour**: Ask questions regarding the colours of objects detected (e.g., "What is the colour of the object?" or "Which colour is the object?").

Ensure that each question is clear, simple, and can be answered with short, concise answers.
Ensure each question is not repeated.
Only include the questions and category.
"""

In [14]:
prompt = f"""
Task:
Generate a list of {questions} SIMPLE questions for the following topic related to object detection for other Visual Language Models that can be easily infered from the input image:

Instructions:
-  **Classes of Object Detection**: Ask questions for the categories or types of objects detected in the image (e.g., "What is the object class?" or "Which class is the object?").
-  The answers to the questions should be specifically object/objects detected in the image 
-  Do not include colours of the objects in the questions
Ensure that each question is clear, simple, and can be answered with short, concise answers.
Ensure each question is not repeated.
The output should only include the questions, Do not include the header.
"""

In [15]:
print(prompt)


Task:
Generate a list of 20 SIMPLE questions for the following topic related to object detection for other Visual Language Models that can be easily infered from the input image:

Instructions:
-  **Classes of Object Detection**: Ask questions for the categories or types of objects detected in the image (e.g., "What is the object class?" or "Which class is the object?").
-  The answers to the questions should be specifically object/objects detected in the image 
-  Do not include colours of the objects in the questions
Ensure that each question is clear, simple, and can be answered with short, concise answers.
Ensure each question is not repeated.
The output should only include the questions, Do not include the header.



In [16]:
import torch
torch.cuda.empty_cache()

In [17]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "url": f"{url}"},
            {"type": "text", "text": f"{prompt}"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=1000)
print(output[0]["generated_text"][-1]["content"])
output_text = output[0]["generated_text"][-1]["content"]
# Okay, let's take a look! 
# Based on the image, the animal on the candy is a **turtle**. 
# You can see the shell shape and the head and legs.

1.  What is the main animal in the image?
2.  What type of clothing is the person wearing?
3.  What is on the ground near the person?
4.  What is the surface the person and dog are sitting on?
5.  What is the body of water visible in the background?
6.  What is the dog wearing?
7.  What is the person holding?
8.  What is the object near the dog's paw?
9.  What is the primary element in the background?
10. What is the general environment depicted?
11. What is the person doing?
12. What is the dog doing?
13. What is the object between the person and the dog?
14. What is the texture of the ground?
15. What is the shape of the waves?
16. What is the light source in the image?
17. What is the main feature of the dog’s appearance?
18. What is the person’s posture?
19. What is the color of the dog’s fur?
20. What is the main element in the foreground?


In [18]:
# File path to save the questions
file_path = '/home/jovyan/Evaluation/questions(OD).txt'

cleaned_questions = '\n'.join([line.split('. ', 1)[1] for line in output_text.splitlines() if line])
# Writing the questions string to a txt file
with open(file_path, 'w') as file:
    file.write(cleaned_questions)  # Strip any leading/trailing whitespace

print(f"Questions have been saved to {file_path}")

Questions have been saved to questions(OD).txt


In [19]:
import re
import json

qa_raw_list = output_text.strip().split("\n")
questions = []
answers = []

# Step 2: Loop through the raw list to extract the questions and answers
for i in range(0, len(qa_raw_list), 3):  # Step by 3 since each question and answer is followed by a blank line
    question = re.sub(r'^\d+\.\s*', '', qa_raw_list[i]).strip()
    questions.append(question)
    answer = qa_raw_list[i + 1].strip().lstrip('* ')  # Clean the answer (remove '* ' prefix)
    answers.append(answer)
    
qa_data = [{"question": question, "answer": answer} for question, answer in zip(questions, answers)]    

file_name= 'questions(OD).json'
# Step 4: Save the dictionary as a JSON file
with open(file_name, 'w') as json_file:
    json.dump(qa_data, json_file, indent=4)

print(f"The question-answer pairs have been saved to {file_name} .")

The question-answer pairs have been saved to questions(OD).json .


In [20]:


# Open and load the JSON file
with open('questions_answers(Gemma3).json', 'r') as file:
    data = json.load(file)

# Print the contents of the JSON file
print(json.dumps(data, indent=4,ensure_ascii=False))  # This will format the JSON for better readability


JSONDecodeError: Expecting value: line 1 column 1 (char 0)